<a href="https://colab.research.google.com/github/arsbremontix/algorithms/blob/main/src/SkipList.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
A toy skip list implementation.
"""

import random

In [1]:
class Node:
    """ A class for a node in the skip list. """
    def __init__(self, value, level):
        self.value = value
        # List of forward pointers
        self.forward = [None] * (level + 1)

class SkipList:
    """ A class for the skip list. """
    def __init__(self, max_level, p):
        # Maximum level for this skip list
        self.max_level = max_level
        # P is the fraction of the nodes with level i pointers also having level i+1 pointers
        self.p = p
        # Create the header node and initialize key to -1
        self.header = self.create_node(self.max_level, float('-inf'))
        # Current level of skip list
        self.level = 0

    def create_node(self, lvl, val):
        return Node(val, lvl)

    def random_level(self):
        """ Randomly decide the level of the node. """
        lvl = 0
        while random.random() < self.p and lvl < self.max_level:
            lvl += 1
        return lvl

    def insert(self, value):
        """ Insert given value in the skip list. """
        update = [None] * (self.max_level + 1)
        current = self.header

        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].value < value:
                current = current.forward[i]
            update[i] = current

        # Reached level 0 and forward pointer to right, which is desired position to insert key
        current = current.forward[0]

        if current is None or current.value != value:
            # Generate a random level for node
            rlevel = self.random_level()

            # If random level is greater than list's current level (node with highest level inserted in list so far)
            if rlevel > self.level:
                for i in range(self.level + 1, rlevel + 1):
                    update[i] = self.header
                self.level = rlevel

            # Create new node with random level generated
            n = self.create_node(rlevel, value)

            # Insert node by rearranging references
            for i in range(rlevel + 1):
                n.forward[i] = update[i].forward[i]
                update[i].forward[i] = n

    def delete(self, value):
        """ Delete element from the skip list. """
        update = [None] * (self.max_level + 1)
        current = self.header

        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].value < value:
                current = current.forward[i]
            update[i] = current

        current = current.forward[0]

        # If current node is target node
        if current and current.value == value:
            for i in range(self.level + 1):
                if update[i].forward[i] != current:
                    break
                update[i].forward[i] = current.forward[i]

            # Remove levels having no elements
            while self.level > 0 and self.header.forward[self.level] is None:
                self.level -= 1

    def search(self, value):
        """ Search for the element in the skip list. """
        current = self.header
        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].value < value:
                current = current.forward[i]
        current = current.forward[0]
        if current and current.value == value:
            return True
        return False

    def display_list(self):
        """ Display the skip list level wise. """
        print("Skip List:")
        for i in range(self.level + 1):
            print("Level {}: ".format(i), end="")
            node = self.header.forward[i]
            while node:
                print(node.value, end=" ")
                node = node.forward[i]
            print("")


In [ ]:
# Example Usage
skip_list = SkipList(3, 0.5)
elements = [3, 6, 7, 9, 12, 19, 17]
for elem in elements:
    skip_list.insert(elem)

skip_list.display_list()
print("Search for 6:", skip_list.search(6))
# print("Search for 15:", skip_list.search(15))

# skip_list.delete(6)
# print("After deleting 6")
# skip_list.display_list()

## References
- https://github.com/google/leveldb/blob/main/db/skiplist.h
- ...